In [1]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import scipy.constants as cts
import pylcp
import time
import random
from tqdm import tqdm
from scipy.interpolate import interp1d
from scipy import stats
from scipy import integrate
from celluloid import Camera
from bayes_opt import BayesianOptimization
import json
import seaborn as sns
import White_class
from scipy import special

In [2]:
# main_detune = 18.7
# sideband_detune = 120
# white_detune = 15
# beta_1 = 0.69
# beta_2 = 1.98
# laseron = 171312
# laseroff = 505566


# test = White_class.Whitelight(main_detune,sideband_detune,white_detune, beta_1,beta_2,laseron,laseroff)

In [3]:
#Main variables

MOT_power = 50
v0_start=1
v0_step=1
v0_end=25
t0_start=0
t0_step=1
t0_end=3500000


#Define the constants
Gamma = 22 # Hz to kHz, Decay rate
wavelength = 359.3e-9 # m to mm
k = 1/wavelength*2*np.pi #x_0
x0 = 1/k
t0 = 1/Gamma*1/(2*np.pi*1e6)
v0 = x0/t0
m0 = cts.hbar*t0/x0**2
a0 = x0/t0**2
F0 = cts.hbar/(x0*t0)
mass = 43*cts.value('atomic mass constant')/m0 # m_0
mag_field_grad = 1252.8168984164048*x0
waist = 0.012/x0
v_max = 20
z_max = 384.855e-3/x0
z_start = 384.855e-3/x0
dz = 0.0005/x0
dv = 0.05
omega = 2*np.pi*(cts.c/wavelength) #Transition frequency
Isat = np.pi*cts.h*cts.c*Gamma*2*np.pi*1e6/3*1/(wavelength)**3
t_eval = np.arange(t0_start,t0_end,t0_step)

# The detunings used in the PRAs:
intensities = 2.*MOT_power*1e-3/(np.pi*0.012**2)/Isat

#Define the hamiltonian
H0_X, Bq_X, U_X, Xbasis = pylcp.hamiltonians.XFmolecules.Xstate(
gamma = 50.697/Gamma,b=154.7/Gamma, c=178.5/Gamma,
    muB = cts.value('Bohr magneton in Hz/T')/1e6*1e-4/Gamma,return_basis=True
    )

# b : SI coupling(isotropic), c : Iz Sz coupling(anisotropic), cc : I N coupling, gamma : S N coupling

E_X = np.unique(np.diag(H0_X))

H0_A, Bq_A, Abasis = pylcp.hamiltonians.XFmolecules.Astate(
    P=+1, Ahfs=-1.5/Gamma, q=0, p=0,gJ=-0.00002,
    muB=cts.value('Bohr magneton in Hz/T')/1e6*1e-4/Gamma, return_basis=True
    )

# gJ : Lande g-factor, p : parity(e parity)

E_A = np.unique(np.diag(H0_A))

dijq = pylcp.hamiltonians.XFmolecules.dipoleXandAstates(
    Xbasis, Abasis, UX=U_X
    )

hamiltonian = pylcp.hamiltonian(H0_X, H0_A, Bq_X, Bq_A, dijq,mass = mass)

In [4]:
#Define the zero magnetic field.
    
magField = pylcp.quadrupoleMagneticField(mag_field_grad)
# magField = np.zeros(3,)

In [5]:
def Fixed_detune_MgF_MOT(main_det,det_1,det_2,beta_1,beta_2,laseron,laseroff):
    det_side = det_1/Gamma
    det_side2 = det_2/Gamma
    Avg_X = np.average(E_X)
    init_pow = 0.5*2./(np.pi*(0.012)**2)/Isat
    
    def Gaussain_Beam_Diagonal(R:np.array,waist):
        return np.exp(-2*((R[0]-R[1])**2/2+R[2]**2)/waist**2)
    
    def Bessel_Intensity(n_order,beta):

        return special.jv(n_order,beta)**2
    
    def Heav_step(t):
        if laseron<=t and t<laseron+14:
            return -1*(t-laseron-7)*((t-laseron-7)**2-49*3)*1/686*1/2+1/2
        elif laseron+14<=t and t<laseroff:
            return 1
        elif t>=laseroff and t<laseroff+14:
            return (t-laseroff-7)*((t-laseroff-7)**2-49*3)*1/686*1/2 + 1/2
        else:
            return 0    
        
    laserBeams = pylcp.laserBeams()
    
    laserBeams += pylcp.laserBeams([{'kvec':np.array([-1,-1,0]),'pol':+1,'pol_coord':'spherical','delta':(E_A[-1]-Avg_X-main_det)-det_side-det_side2*2,
                                     's': lambda R,t : init_pow*Gaussain_Beam_Diagonal(R,waist)*Heav_step(t)*Bessel_Intensity(-1,beta_1)*Bessel_Intensity(2,beta_2)},
                                    {'kvec':np.array([-1,-1,0]),'pol':-1,'pol_coord':'spherical','delta':(E_A[-1]-Avg_X-main_det)-det_side-det_side2*2,
                                     's': lambda R,t : init_pow*Gaussain_Beam_Diagonal(R,waist)*Heav_step(t)*Bessel_Intensity(-1,beta_1)*Bessel_Intensity(2,beta_2)}])
    laserBeams += pylcp.laserBeams([{'kvec':np.array([-1,-1,0]),'pol':+1,'pol_coord':'spherical','delta':(E_A[-1]-Avg_X-main_det)-det_side-det_side2*1,
                                     's': lambda R,t : init_pow*Gaussain_Beam_Diagonal(R,waist)*Heav_step(t)*Bessel_Intensity(-1,beta_1)*Bessel_Intensity(1,beta_2)},
                                    {'kvec':np.array([-1,-1,0]),'pol':-1,'pol_coord':'spherical','delta':(E_A[-1]-Avg_X-main_det)-det_side-det_side2*1,
                                     's': lambda R,t : init_pow*Gaussain_Beam_Diagonal(R,waist)*Heav_step(t)*Bessel_Intensity(-1,beta_1)*Bessel_Intensity(1,beta_2)}])
    laserBeams += pylcp.laserBeams([{'kvec':np.array([-1,-1,0]),'pol':+1,'pol_coord':'spherical','delta':(E_A[-1]-Avg_X-main_det)-det_side-det_side2*0,
                                     's': lambda R,t : init_pow*Gaussain_Beam_Diagonal(R,waist)*Heav_step(t)*Bessel_Intensity(-1,beta_1)*Bessel_Intensity(0,beta_2)},
                                    {'kvec':np.array([-1,-1,0]),'pol':-1,'pol_coord':'spherical','delta':(E_A[-1]-Avg_X-main_det)-det_side-det_side2*0,
                                     's': lambda R,t : init_pow*Gaussain_Beam_Diagonal(R,waist)*Heav_step(t)*Bessel_Intensity(-1,beta_1)*Bessel_Intensity(0,beta_2)}])
    laserBeams += pylcp.laserBeams([{'kvec':np.array([-1,-1,0]),'pol':+1,'pol_coord':'spherical','delta':(E_A[-1]-Avg_X-main_det)-det_side+det_side2*1,
                                     's': lambda R,t : init_pow*Gaussain_Beam_Diagonal(R,waist)*Heav_step(t)*Bessel_Intensity(-1,beta_1)*Bessel_Intensity(-1,beta_2)},
                                    {'kvec':np.array([-1,-1,0]),'pol':-1,'pol_coord':'spherical','delta':(E_A[-1]-Avg_X-main_det)-det_side+det_side2*1,
                                     's': lambda R,t : init_pow*Gaussain_Beam_Diagonal(R,waist)*Heav_step(t)*Bessel_Intensity(-1,beta_1)*Bessel_Intensity(-1,beta_2)}])
    laserBeams += pylcp.laserBeams([{'kvec':np.array([-1,-1,0]),'pol':+1,'pol_coord':'spherical','delta':(E_A[-1]-Avg_X-main_det)-det_side+det_side2*2,
                                     's': lambda R,t : init_pow*Gaussain_Beam_Diagonal(R,waist)*Heav_step(t)*Bessel_Intensity(-1,beta_1)*Bessel_Intensity(-2,beta_2)},
                                    {'kvec':np.array([-1,-1,0]),'pol':-1,'pol_coord':'spherical','delta':(E_A[-1]-Avg_X-main_det)-det_side+det_side2*2,
                                     's': lambda R,t : init_pow*Gaussain_Beam_Diagonal(R,waist)*Heav_step(t)*Bessel_Intensity(-1,beta_1)*Bessel_Intensity(-2,beta_2)}])

# Main Slowing Laser
    laserBeams += pylcp.laserBeams([{'kvec':np.array([-1,-1,0]),'pol':+1,'pol_coord':'spherical','delta':(E_A[-1]-Avg_X-main_det)-det_side2*2,
                                     's': lambda R,t : init_pow*Gaussain_Beam_Diagonal(R,waist)*Heav_step(t)*Bessel_Intensity(0,beta_1)*Bessel_Intensity(2,beta_2)},
                                    {'kvec':np.array([-1,-1,0]),'pol':-1,'pol_coord':'spherical','delta':(E_A[-1]-Avg_X-main_det)-det_side2*2,
                                     's': lambda R,t : init_pow*Gaussain_Beam_Diagonal(R,waist)*Heav_step(t)*Bessel_Intensity(0,beta_1)*Bessel_Intensity(2,beta_2)}])
    laserBeams += pylcp.laserBeams([{'kvec':np.array([-1,-1,0]),'pol':+1,'pol_coord':'spherical','delta':(E_A[-1]-Avg_X-main_det)-det_side2*1,
                                     's': lambda R,t : init_pow*Gaussain_Beam_Diagonal(R,waist)*Heav_step(t)*Bessel_Intensity(0,beta_1)*Bessel_Intensity(1,beta_2)},
                                    {'kvec':np.array([-1,-1,0]),'pol':-1,'pol_coord':'spherical','delta':(E_A[-1]-Avg_X-main_det)-det_side2*1,
                                     's': lambda R,t : init_pow*Gaussain_Beam_Diagonal(R,waist)*Heav_step(t)*Bessel_Intensity(0,beta_1)*Bessel_Intensity(1,beta_2)}])
    laserBeams += pylcp.laserBeams([{'kvec':np.array([-1,-1,0]),'pol':+1,'pol_coord':'spherical','delta':(E_A[-1]-Avg_X-main_det)-det_side2*0,
                                     's': lambda R,t : init_pow*Gaussain_Beam_Diagonal(R,waist)*Heav_step(t)*Bessel_Intensity(0,beta_1)*Bessel_Intensity(0,beta_2)},
                                    {'kvec':np.array([-1,-1,0]),'pol':-1,'pol_coord':'spherical','delta':(E_A[-1]-Avg_X-main_det)-det_side2*0,
                                     's': lambda R,t : init_pow*Gaussain_Beam_Diagonal(R,waist)*Heav_step(t)*Bessel_Intensity(0,beta_1)*Bessel_Intensity(0,beta_2)}])
    laserBeams += pylcp.laserBeams([{'kvec':np.array([-1,-1,0]),'pol':+1,'pol_coord':'spherical','delta':(E_A[-1]-Avg_X-main_det)+det_side2*1,
                                     's': lambda R,t : init_pow*Gaussain_Beam_Diagonal(R,waist)*Heav_step(t)*Bessel_Intensity(0,beta_1)*Bessel_Intensity(-1,beta_2)},
                                    {'kvec':np.array([-1,-1,0]),'pol':-1,'pol_coord':'spherical','delta':(E_A[-1]-Avg_X-main_det)+det_side2*1,
                                     's': lambda R,t : init_pow*Gaussain_Beam_Diagonal(R,waist)*Heav_step(t)*Bessel_Intensity(0,beta_1)*Bessel_Intensity(-1,beta_2)}])
    laserBeams += pylcp.laserBeams([{'kvec':np.array([-1,-1,0]),'pol':+1,'pol_coord':'spherical','delta':(E_A[-1]-Avg_X-main_det)+det_side2*2,
                                     's': lambda R,t : init_pow*Gaussain_Beam_Diagonal(R,waist)*Heav_step(t)*Bessel_Intensity(0,beta_1)*Bessel_Intensity(-2,beta_2)},
                                    {'kvec':np.array([-1,-1,0]),'pol':-1,'pol_coord':'spherical','delta':(E_A[-1]-Avg_X-main_det)+det_side2*2,
                                     's': lambda R,t : init_pow*Gaussain_Beam_Diagonal(R,waist)*Heav_step(t)*Bessel_Intensity(0,beta_1)*Bessel_Intensity(-2,beta_2)}])
# Plus Sideband part
    laserBeams += pylcp.laserBeams([{'kvec':np.array([-1,-1,0]),'pol':+1,'pol_coord':'spherical','delta':(E_A[-1]-Avg_X-main_det)+det_side-det_side2*2,
                                     's': lambda R,t : init_pow*Gaussain_Beam_Diagonal(R,waist)*Heav_step(t)*Bessel_Intensity(1,beta_1)*Bessel_Intensity(2,beta_2)},
                                    {'kvec':np.array([-1,-1,0]),'pol':-1,'pol_coord':'spherical','delta':(E_A[-1]-Avg_X-main_det)+det_side-det_side2*2,
                                     's': lambda R,t : init_pow*Gaussain_Beam_Diagonal(R,waist)*Heav_step(t)*Bessel_Intensity(1,beta_1)*Bessel_Intensity(2,beta_2)}])
    laserBeams += pylcp.laserBeams([{'kvec':np.array([-1,-1,0]),'pol':+1,'pol_coord':'spherical','delta':(E_A[-1]-Avg_X-main_det)+det_side-det_side2*1,
                                     's': lambda R,t : init_pow*Gaussain_Beam_Diagonal(R,waist)*Heav_step(t)*Bessel_Intensity(1,beta_1)*Bessel_Intensity(1,beta_2)},
                                    {'kvec':np.array([-1,-1,0]),'pol':-1,'pol_coord':'spherical','delta':(E_A[-1]-Avg_X-main_det)+det_side-det_side2*1,
                                     's': lambda R,t : init_pow*Gaussain_Beam_Diagonal(R,waist)*Heav_step(t)*Bessel_Intensity(1,beta_1)*Bessel_Intensity(1,beta_2)}])
    laserBeams += pylcp.laserBeams([{'kvec':np.array([-1,-1,0]),'pol':+1,'pol_coord':'spherical','delta':(E_A[-1]-Avg_X-main_det)+det_side-det_side2*0,
                                     's': lambda R,t : init_pow*Gaussain_Beam_Diagonal(R,waist)*Heav_step(t)*Bessel_Intensity(1,beta_1)*Bessel_Intensity(0,beta_2)},
                                    {'kvec':np.array([-1,-1,0]),'pol':-1,'pol_coord':'spherical','delta':(E_A[-1]-Avg_X-main_det)+det_side-det_side2*0,
                                     's': lambda R,t : init_pow*Gaussain_Beam_Diagonal(R,waist)*Heav_step(t)*Bessel_Intensity(1,beta_1)*Bessel_Intensity(0,beta_2)}])
    laserBeams += pylcp.laserBeams([{'kvec':np.array([-1,-1,0]),'pol':+1,'pol_coord':'spherical','delta':(E_A[-1]-Avg_X-main_det)+det_side+det_side2*1,
                                     's': lambda R,t : init_pow*Gaussain_Beam_Diagonal(R,waist)*Heav_step(t)*Bessel_Intensity(1,beta_1)*Bessel_Intensity(-1,beta_2)},
                                    {'kvec':np.array([-1,-1,0]),'pol':-1,'pol_coord':'spherical','delta':(E_A[-1]-Avg_X-main_det)+det_side+det_side2*1,
                                     's': lambda R,t : init_pow*Gaussain_Beam_Diagonal(R,waist)*Heav_step(t)*Bessel_Intensity(1,beta_1)*Bessel_Intensity(-1,beta_2)}])
    laserBeams += pylcp.laserBeams([{'kvec':np.array([-1,-1,0]),'pol':+1,'pol_coord':'spherical','delta':(E_A[-1]-Avg_X-main_det)+det_side+det_side2*2,
                                     's': lambda R,t : init_pow*Gaussain_Beam_Diagonal(R,waist)*Heav_step(t)*Bessel_Intensity(1,beta_1)*Bessel_Intensity(-2,beta_2)},
                                    {'kvec':np.array([-1,-1,0]),'pol':-1,'pol_coord':'spherical','delta':(E_A[-1]-Avg_X-main_det)+det_side+det_side2*2,
                                     's': lambda R,t : init_pow*Gaussain_Beam_Diagonal(R,waist)*Heav_step(t)*Bessel_Intensity(1,beta_1)*Bessel_Intensity(-2,beta_2)}])
    return laserBeams

In [6]:
def slow_bayesian(main_det,det_1,det_2,beta_1,beta_2,laseron,laseroff):
    laserBeams = Fixed_detune_MgF_MOT(main_det,det_1,det_2,beta_1,beta_2,laseron,laseroff)
    magField = pylcp.quadrupoleMagneticField(mag_field_grad)
    rateeq = pylcp.rateeq(laserBeams=laserBeams,magField=magField,hamitlonian=hamiltonian)

    
    def Capture_velocity_condition(t,y,threshold = 5.810):
        if y[-6]<threshold:
            val = -1.
        else:
            val = 1.
        return val
    def Lost_condition(t,y,threshold = 0.):
        if y[-6]<threshold:
            val = -1.
        else:
            val = 1.
        return val
    def for_transverse_condition(t,y,threshold = -0.020/x0):
        if y[-3]>threshold:
            val = -1.
        else:
            val = 1.
        return val

    Capture_velocity_condition.terminal = False
    Lost_condition.terminal = False
    for_transverse_condition.terminal = False
    conditions =  [for_transverse_condition,Lost_condition,Capture_velocity_condition]
    
    v_longitudinal = np.linspace(14,21,16)
    time_final = list()
    v_trap_initial = list()
    
    for v0_longitudinal in v_longitudinal:
        rateeq.set_initial_position_and_velocity(np.array([-1*z_start/np.sqrt(2),-1*z_start/np.sqrt(2),0]),np.array([v0_longitudinal/np.sqrt(2),v0_longitudinal/np.sqrt(2),0]))
        rateeq.set_initial_pop(np.array([1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0]))

        rateeq.evolve_motion([0.,max(t_eval)],t_eval=t_eval,events= conditions,max_step=1e5,progress_bar = 0,method='LSODA')
        sol = rateeq.sol
        # print(sol.t_events)
        if len(sol.t_events[0])==1:
            if len(sol.t_events[1])==0:
                if len(sol.t_events[2])==1:
                    time_final.append(sol.t_events[0][0])
                    v_trap_initial.append(sol.v[0][0])
            
            
                    
    # print(v_trap_initial)
    # print(time_final)
                    
    if len(time_final)<=1:
        # print("NO results")
        return 0.
    
    def time_vs_v_final(vff):
        if vff>= min(v_trap_initial) and vff<=max(v_trap_initial):
            return np.interp(vff,v_trap_initial,time_final)

    def v_transverse_vs_v_traverse(vfs):
        return (6.00/time_vs_v_final(vfs)*1e-3/t0)

    muf = 140/v0
    sigf = 17/v0
    mut = 0
    sigt = 18.7564/v0

    def Transverse_percentage(v):
        rv = stats.norm(mut,sigt) # Gaussian of transverse, v0 scale
        val = (rv.cdf(v_transverse_vs_v_traverse(v))-rv.cdf(0))/rv.cdf(sigt*2)*2

        if val >= 1:
            return 1
        else:
            return val

    def total_func(v):
        rv_f = stats.norm(muf,sigf) # Gaussian of forward, v0 scale
        return Transverse_percentage(v)*rv_f.pdf(v)

    return integrate.quad(total_func,v_trap_initial[0],v_trap_initial[-1],limit=100)[0]

In [7]:
start = time.time()

print(slow_bayesian(18.7,120,15,0.69,1.98,171312,505566))

print(time.time()-start)

0.04233822160541753
226.70168447494507


In [8]:
pbounds = {'main_det':(10,20),'det_1':(70,170),'det_2':(5,30),'beta_1':(0,2),'beta_2':(0,5),'laseron':(138240,300000),'laseroff':(300001,600000)}

In [9]:
optimizer = BayesianOptimization(
    f = slow_bayesian,
    pbounds=pbounds,
    verbose=13,
    random_state=1,
)

In [10]:
from bayes_opt.util import load_logs

# New optimizer is loaded with previously seen points
load_logs(optimizer, logs=["./logs_Expected_model.json"])

In [11]:
start = time.time()
optimizer.maximize(
    init_points=1,
    n_iter=1,)
print(time.time()-start)

|   iter    |  target   |  beta_1   |  beta_2   |   det_1   |   det_2   | laseroff  |  laseron  | main_det  |
-------------------------------------------------------------------------------------------------------------
|  1        |  0.0      |  0.834    |  3.602    |  70.01    |  12.56    |  3.44e+05 |  1.532e+0 |  11.86    |
|  2        |  0.0      |  1.059    |  2.593    |  155.1    |  14.86    |  3.28e+05 |  2.377e+0 |  10.4     |
243.21982312202454


In [12]:
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events

logger = JSONLogger(path='./logs_Expected_model.json',reset=True)
optimizer.subscribe(Events.OPTIMIZATION_STEP,logger)

In [13]:
start = time.time()
optimizer.maximize(
    init_points=5,
    n_iter=10,)
print(time.time()-start)

|   iter    |  target   |  beta_1   |  beta_2   |   det_1   |   det_2   | laseroff  |  laseron  | main_det  |
-------------------------------------------------------------------------------------------------------------
|  3        |  0.0      |  1.059    |  2.593    |  155.1    |  14.86    |  3.28e+05 |  2.377e+0 |  10.4     |
|  4        |  0.0      |  1.059    |  2.593    |  155.1    |  14.86    |  3.28e+05 |  2.377e+0 |  10.4     |
|  5        |  0.0      |  1.059    |  2.593    |  155.1    |  14.86    |  3.28e+05 |  2.377e+0 |  10.4     |
|  6        |  0.0      |  1.059    |  2.593    |  155.1    |  14.86    |  3.28e+05 |  2.377e+0 |  10.4     |
|  7        |  0.0      |  1.059    |  2.593    |  155.1    |  14.86    |  3.28e+05 |  2.377e+0 |  10.4     |
|  8        |  0.06112  |  0.9662   |  2.856    |  74.6     |  20.33    |  3.706e+0 |  1.961e+0 |  18.75    |
|  9        |  0.009816 |  0.8555   |  1.586    |  136.1    |  23.06    |  4.521e+0 |  2.5e+05  |  12.54    |


C:\Users\qmopl\AppData\Local\Temp/ipykernel_10676/1716087149.py:82: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  return integrate.quad(total_func,v_trap_initial[0],v_trap_initial[-1],limit=100)[0]


|  10       |  0.04788  |  1.909    |  0.3369   |  114.3    |  11.51    |  4.403e+0 |  2.652e+0 |  16.59    |
|  11       |  0.002823 |  0.7652   |  3.3      |  122.8    |  22.32    |  5.223e+0 |  2.171e+0 |  11.89    |
|  12       |  0.0      |  0.6823   |  4.563    |  136.5    |  18.79    |  3.454e+0 |  2.755e+0 |  18.21    |
|  13       |  0.0      |  1.564    |  1.984    |  71.55    |  26.1     |  5.86e+05 |  2.271e+0 |  11.83    |
|  14       |  0.007634 |  0.8455   |  4.331    |  150.2    |  7.735    |  4.172e+0 |  2.483e+0 |  16.96    |
|  15       |  0.05951  |  0.8455   |  4.022    |  165.1    |  18.93    |  5.529e+0 |  1.662e+0 |  18.09    |
|  16       |  0.0      |  0.6251   |  2.566    |  83.36    |  12.18    |  3.259e+0 |  2.712e+0 |  11.47    |
|  17       |  0.06592  |  0.7024   |  0.5903   |  137.0    |  27.75    |  4.779e+0 |  2.881e+0 |  16.86    |
2561.2816123962402


In [14]:
start = time.time()
optimizer.maximize(
    init_points=10,
    n_iter=100,)
print(time.time()-start)

|   iter    |  target   |  beta_1   |  beta_2   |   det_1   |   det_2   | laseroff  |  laseron  | main_det  |
-------------------------------------------------------------------------------------------------------------
|  18       |  0.06592  |  0.7024   |  0.5903   |  137.0    |  27.75    |  4.779e+0 |  2.881e+0 |  16.86    |
|  19       |  0.06592  |  0.7024   |  0.5903   |  137.0    |  27.75    |  4.779e+0 |  2.881e+0 |  16.86    |
|  20       |  0.06592  |  0.7024   |  0.5903   |  137.0    |  27.75    |  4.779e+0 |  2.881e+0 |  16.86    |
|  21       |  0.06592  |  0.7024   |  0.5903   |  137.0    |  27.75    |  4.779e+0 |  2.881e+0 |  16.86    |
|  22       |  0.06592  |  0.7024   |  0.5903   |  137.0    |  27.75    |  4.779e+0 |  2.881e+0 |  16.86    |
|  23       |  0.06592  |  0.7024   |  0.5903   |  137.0    |  27.75    |  4.779e+0 |  2.881e+0 |  16.86    |
|  24       |  0.06592  |  0.7024   |  0.5903   |  137.0    |  27.75    |  4.779e+0 |  2.881e+0 |  16.86    |
|  25     

C:\Users\qmopl\AppData\Local\Temp/ipykernel_10676/1716087149.py:82: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  return integrate.quad(total_func,v_trap_initial[0],v_trap_initial[-1],limit=100)[0]


|  32       |  0.06581  |  0.4743   |  2.38     |  98.22    |  28.7     |  5.705e+0 |  2.784e+0 |  18.14    |
|  33       |  0.03796  |  1.265    |  2.832    |  146.2    |  6.347    |  4.927e+0 |  2.601e+0 |  14.77    |
|  34       |  0.05823  |  0.061    |  2.255    |  88.64    |  7.45     |  5.193e+0 |  1.751e+0 |  18.12    |
|  35       |  0.000144 |  1.739    |  2.135    |  95.12    |  13.24    |  3.785e+0 |  2.72e+05 |  11.21    |
|  36       |  0.04169  |  1.343    |  0.499    |  126.0    |  22.68    |  5.022e+0 |  2.439e+0 |  16.65    |
|  37       |  0.01743  |  1.707    |  3.056    |  71.92    |  13.21    |  4.106e+0 |  2.88e+05 |  18.9     |
|  38       |  0.0      |  0.03977  |  4.478    |  158.7    |  11.53    |  3.256e+0 |  1.445e+0 |  12.45    |
|  39       |  0.02363  |  0.2497   |  0.1445   |  131.6    |  23.87    |  3.845e+0 |  2.342e+0 |  16.76    |
|  40       |  0.0      |  0.4628   |  4.623    |  121.1    |  11.96    |  3.746e+0 |  2.708e+0 |  14.08    |
|  41     

C:\Users\qmopl\AppData\Local\Temp/ipykernel_10676/1716087149.py:82: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  return integrate.quad(total_func,v_trap_initial[0],v_trap_initial[-1],limit=100)[0]


|  88       |  0.04478  |  0.6092   |  3.742    |  144.6    |  26.13    |  4.61e+05 |  1.531e+0 |  16.21    |


...

C:\Users\qmopl\AppData\Local\Temp/ipykernel_10676/1716087149.py:82: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  return integrate.quad(total_func,v_trap_initial[0],v_trap_initial[-1],limit=100)[0]


|  91       |  0.04759  |  0.1867   |  2.612    |  162.9    |  22.91    |  4.537e+0 |  2.421e+0 |  17.02    |


C:\Users\qmopl\AppData\Local\Temp/ipykernel_10676/1716087149.py:82: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  return integrate.quad(total_func,v_trap_initial[0],v_trap_initial[-1],limit=100)[0]


|  92       |  0.0649   |  1.306    |  2.867    |  126.0    |  10.03    |  4.649e+0 |  2.682e+0 |  17.48    |
|  93       |  0.0017   |  0.8638   |  3.722    |  76.37    |  12.04    |  4.328e+0 |  1.646e+0 |  12.93    |
|  94       |  0.01751  |  0.6035   |  1.795    |  74.86    |  27.76    |  4.926e+0 |  2.136e+0 |  13.27    |
|  95       |  0.02347  |  0.7274   |  0.484    |  125.6    |  26.72    |  4.05e+05 |  2.516e+0 |  15.55    |
|  96       |  0.0307   |  0.3547   |  2.597    |  165.7    |  14.22    |  5.168e+0 |  1.419e+0 |  13.67    |
|  97       |  0.02045  |  1.245    |  3.104    |  106.2    |  12.06    |  5.053e+0 |  2.304e+0 |  13.89    |
|  98       |  0.0      |  1.656    |  4.505    |  120.0    |  17.1     |  3.295e+0 |  2.469e+0 |  16.01    |
|  99       |  0.06171  |  1.167    |  3.156    |  158.0    |  26.47    |  4.506e+0 |  2.742e+0 |  19.57    |
|  100      |  0.0      |  1.849    |  0.1891   |  126.3    |  21.68    |  5.493e+0 |  2.059e+0 |  14.94    |
|  101    

C:\Users\qmopl\AppData\Local\Temp/ipykernel_10676/1716087149.py:82: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  return integrate.quad(total_func,v_trap_initial[0],v_trap_initial[-1],limit=100)[0]
